In [44]:
!pip install clean-text
!pip install inflect

^C



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [38]:
import numpy as np
import pandas as pd
import re
import inflect
from cleantext import clean
import json
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import random
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from Spelling_correction import find_closest_match


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [39]:
def load_data(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            data.append(line.strip())  # Use strip() to remove leading/trailing whitespace
    return data

In [40]:
path='input.txt'
test_data=load_data(path)
print(len(test_data))


2


In [41]:
# Create an inflect engine
p = inflect.engine()

def replace_numbers_with_words(text):
    # Regular expression to find numbers in the text
    number_pattern = r'\b\d+\b'
    
    # Function to convert a number match to its word representation
    def number_to_words(match):
        number = int(match.group())
        # Convert to words and replace hyphens with underscores
        return p.number_to_words(number, andword="").replace('-', '_')

    
    # Substitute numbers with their word representation
    text = re.sub(number_pattern, number_to_words, text)
    
    # Replace standalone 'a' or 'an' with 'one' where appropriate
    text = re.sub(
        r'\b(a|an)\b(?! ((lot)|(little)|(few)|(number)|(couple)|(bit)|(load)|(stack)|(bunch)|(group)|(set)|(series)|(variety)|(range)|(amount)|(sum)|(total)))',
       'one',
        text,
        flags=re.IGNORECASE
    )
    return text

def replace_numbers_with_words2(text):
    # Regular expression to find numbers in the text
    number_pattern = r'\b\d+\b'
    
    # Function to convert a number match to its word representation
    def number_to_words(match):
        number = int(match.group())
        # Convert to words and replace hyphens with underscores
        return p.number_to_words(number, andword="").replace('-', '_')

    
    # Substitute numbers with their word representation
    text = re.sub(number_pattern, number_to_words, text)

def handle_negations(text):
    #! negations are based on test set
    negations_pattern = r"\b(?:no|not|without)\s+.*?\b(?=(?:[^\w\s]|$))"
    # print(re.findall(negations_pattern, text))
    text = re.sub(negations_pattern, lambda x: ' '.join([f'not_{word}' for word in x.group(0).split()]), text)
    return text

#!# lemmatize words with all possible pos tags
def lem_word(word):
    if(word=='bit'):
        return word
    possible_pos = [wordnet.NOUN, wordnet.VERB, wordnet.ADJ, wordnet.ADV]
    lemmatizer = WordNetLemmatizer()
    for pos in possible_pos:
        word=lemmatizer.lemmatize(word,pos)
    return word


In [42]:
lemmatizer = WordNetLemmatizer()
stopwords = set(stopwords.words('english'))
stopwords.add('like')
not_stopwords = ['not_' + word for word in stopwords]
stopwords=set()

In [ ]:
pretrained_model = gs.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
def process_sentence(sentence, model):
    for i, word in enumerate(sentence):
        if word not in model:
            sentence[i] = 'unk'
    return sentence

data = [process_sentence(sentence, pretrained_model) for sentence in data]
print(data[:5])
vocab=set()
for sentence in data:
    vocab.update(sentence)
#! get word index for each word in vocab
word2idx = {word: idx for idx, word in enumerate(vocab)}

In [43]:
#! tokenize test data and remove stop words
def preprocess_test_data(test_data, stopwords,with_a=True):
    
    test_data=[replace_numbers_with_words(order) for order in test_data] if with_a else [replace_numbers_with_words2(order) for order in test_data]
    test_data = [word_tokenize(order) for order in test_data]
    test_data = [[word.lower() for word in order if word.lower() not in stopwords] for order in test_data]
    test_data = [clean(order, no_line_breaks=True, no_punct=True, no_currency_symbols=True) for order in test_data]
    test_data = [re.sub(r'\bd\s+', '', order) for order in test_data]
    test_data = [word_tokenize(order) for order in test_data]
    test_data = [[lem_word(word) for word in order] for order in test_data]
    return test_data
order_tokens_1 = preprocess_test_data(test_data, stopwords)
order_tokens_2 = preprocess_test_data(test_data, stopwords,with_a=False)


[['i', 'would', 'like', 'to', 'order', 'two', 'large', 'chicago', 'style', 'pizza', 'with', 'extra', 'cheese', 'pepperoni', 'no', 'onion', 'and', 'bacon'], ['and', 'three', 'small', 'thin', 'crust', 'pizza', 'with', 'a', 'lot', 'of', 'olive', 'and', 'ham', 'i', 'also', 'want', 'five', 'large', 'bottle', 'of', 'coke', 'and', 'two', 'medium', 'can', 'of', 'diet', 'pepsi']]


In [ ]:
# Load  model1
order_category_labels=[] # model 1 output

In [ ]:
test_data,test_labels_order_category=order_tokens_1,order_category_labels
print(test_data)
print(test_labels_order_category)
test_data_copy = copy.deepcopy(test_data)  # Deep copy of dev_data
for tokens in test_data:
    for i,word in enumerate(tokens):
        if word not in vocab:
            tokens[i] = 'unk'
X_test=[[word2idx[word] for word in sentence] for sentence in test_data]
X_test=pad_sequences(X_test, maxlen=max_length, padding='post', value=-1)
X_categories_test = [[category for category in sentence_categories] for sentence_categories in test_labels_order_category]
X_categories_test = pad_sequences(X_categories_test, maxlen=max_length, padding='post', value=2)

In [ ]:
model2=load_model('model2.h5')

In [ ]:
pred_test = model2.predict([X_test, X_categories_test])  # Pass both inputs as a list
pred_test = np.argmax(pred_test, axis=-1)  # Get the class with the highest probability
# print(pred_test[:5])
pred_test = [seq[:len(test_data[i])] for i, seq in enumerate(pred_test)]  # Trim each sequence
pred_test = [list(seq) for seq in pred_test] 

In [ ]:
# tokens = ['can', 'you', 'please', 'order', 'one', 'large', 'pepperoni', 'pizza', 'and', 'one', 'medium', 'sausage', 'pizza', 'with', 'one', 'sprite', 'and', 'one', 'coke']
# labels = [0,0,0,0,1,2,4,0,0,1,2,4,0,0,1,8,0,1,8]
# tokens=['two', 'pizzes', 'one', 'with', 'pepproni', 'and','one','with','chicken']
# labels=[1,0,1,0,4,0,1,0,4]
# tokens=['want','one','pizza', 'with', 'pepproni', 'and','with','large','size','iced','tea']
# labels=[0,1,0,0,4,0,0,2,2,8,8]
def combine_tokens(tokens, labels):
    combined_tokens = []
    combined_labels = []
    i = 0

    while i < len(tokens):
        if labels[i] != 0 and labels[i]!=4:  # Check for non-None labels
            # Start combining consecutive tokens with the same label
            combined_token = tokens[i]
            current_label = labels[i]
            i += 1

            while i < len(tokens) and labels[i] == current_label:
                combined_token += f"_{tokens[i]}"
                i += 1

            combined_tokens.append(combined_token)
            combined_labels.append(current_label)
        else:
            # Keep tokens with label 0 as is
            combined_tokens.append(tokens[i])
            combined_labels.append(labels[i])
            i += 1

    return combined_tokens, combined_labels

In [ ]:
label_mapping = {
    0: "NONE",
    1: "NUMBER",
    2: "SIZE",
    3: "STYLE",
    4: "TOPPING",
    5: "QUANTITY",
    6: "NOT_TOPPING",
    7: "NOT_STYLE",
    8: "DRINKTYPE",
    9: "CONTAINERTYPE",
    10: "VOLUME",
}

def Get_EXR_Format(tokens, labels):
    # Step 1: Split into suborders
    suborders = []
    current_suborder = {
        "tokens": [],
        "labels": [],
        "has_number": False,
        "has_size": False,
        "has_style": False,
        "has_drinktype": False,
        "has_volume": False,
        "has_containertype": False
    }
    
    for token, label in zip(tokens, labels):
        if label == 1 and current_suborder["tokens"]:  # New suborder starts at 'Number'
            suborders.append(current_suborder)
            current_suborder = {
                "tokens": [],
                "labels": [],
                "has_number": False,
                "has_size": False,
                "has_style": False,
                "has_drinktype": False,
                "has_volume": False,
                "has_containertype": False
            }
        
        # If the label is SIZE and we already have a size, start a new suborder
        if label == 2 and current_suborder["has_size"] and current_suborder["tokens"]:
            suborders.append(current_suborder)
            current_suborder = {
                "tokens": [],
                "labels": [],
                "has_number": False,
                "has_size": False,
                "has_style": False,
                "has_drinktype": False,
                "has_volume": False,
                "has_containertype": False
            }
        
        # If the label is STYLE and we already have a style, start a new suborder
        if label == 3 and current_suborder["has_style"] and current_suborder["tokens"]:
            suborders.append(current_suborder)
            current_suborder = {
                "tokens": [],
                "labels": [],
                "has_number": False,
                "has_size": False,
                "has_style": False,
                "has_drinktype": False,
                "has_volume": False,
                "has_containertype": False
            }
        if label == 8 and current_suborder["has_drinktype"] and current_suborder["tokens"]:
            suborders.append(current_suborder)
            current_suborder = {
                "tokens": [],
                "labels": [],
                "has_number": False,
                "has_size": False,
                "has_style": False,
                "has_drinktype": False,
                "has_volume": False,
                "has_containertype": False
            }
        if label == 10 and current_suborder["has_volume"] and current_suborder["tokens"]:
            suborders.append(current_suborder)
            current_suborder = {
                "tokens": [],
                "labels": [],
                "has_number": False,
                "has_size": False,
                "has_style": False,
                "has_drinktype": False,
                "has_volume": False,
                "has_containertype": False
            }
        if label == 9 and current_suborder["has_containertype"] and current_suborder["tokens"]:
            suborders.append(current_suborder)
            current_suborder = {
                "tokens": [],
                "labels": [],
                "has_number": False,
                "has_size": False,
                "has_style": False,
                "has_drinktype": False,
                "has_volume": False,
                "has_containertype": False
            }
        
        current_suborder["tokens"].append(token)
        current_suborder["labels"].append(label)

        # Update flags for size and style
        if label == 2:  # SIZE label
            current_suborder["has_size"] = True
        elif label == 3:  # STYLE label
            current_suborder["has_style"] = True
        elif label == 1:  # NUMBER label
            current_suborder["has_number"] = True
        elif label == 8:  # DRINKTYPE label
            current_suborder["has_drinktype"] = True
        elif label == 10:  # VOLUME label
            current_suborder["has_volume"] = True
        elif label == 9:  # CONTAINERTYPE label
            current_suborder["has_containertype"] = True
    
    # Append the last suborder
    if current_suborder["tokens"]:
        suborders.append(current_suborder)
    
    # Step 2: Add default Number if no Number found
    for suborder in suborders:
        if not suborder["has_number"]:
            suborder["tokens"].insert(0, "one")
            suborder["labels"].insert(0, 1)  # Number label
    
    # Step 3: Classify suborders
    order_details = []
    for suborder in suborders:
        tokens = suborder["tokens"]
        labels = suborder["labels"]
        # Determine order type
        if any(label in [8, 9, 10] for label in labels):  # Drink order
            order_type = "DRINKORDER"
        elif any(label in [3, 4, 5, 6, 7] for label in labels):  # Pizza order
            order_type = "PIZZAORDER"
        else:
            continue  # Skip invalid suborders
        
        suborder_details = []
        i = 0
        while i < len(tokens):
            token = tokens[i]
            label = labels[i]
            if label == 5:  # Quantity
                if i != len(tokens) - 1:
                    suborder_details.append(
                        f"(COMPLEX_TOPPING (QUANTITY {token.upper()}) (TOPPING {tokens[i + 1].upper()}))"
                    )
                    i += 1
                else:
                    suborder_details.append(f"({label_mapping[4]} {token.upper()})")
            elif label in [6, 7]:  # NOT cases (NOT_TOPPING, NOT_STYLE)
                new_label = label_mapping[label][4:]  # Remove "NOT_" from the label string
                suborder_details.append(
                    f"(NOT ({new_label} {token.upper()}))"
                )
            elif label != 0:
                # General case for non-zero labels
                suborder_details.append(f"({label_mapping[label]} {token.upper()})")
            i += 1
        
        order_details.append(f"({order_type} {' '.join(suborder_details)})")
    
    # Step 4: Combine into final order
    return f"(ORDER {' '.join(order_details)})"

In [ ]:
def parse_to_dict(s):
    def parse(tokens):
        token = tokens.pop(0)
        if token == '(':
            key = tokens.pop(0)
            nested = {}
            while len(tokens) > 0 and tokens[0] != ')':
                if tokens[0] == '(':
                    temp_dict=parse(tokens)
                    for k,v in temp_dict.items():
                        if k in nested:
                            nested[k].append(v)
                        elif k == 'DRINKORDER' or k == 'PIZZAORDER' or k =='TOPPING' or k =='NOT' or k =='COMPLEX_TOPPING' or k == 'STYLE':
                            nested[k]=[v]
                        else:
                            nested[k] = v
                else:
                    sub_key = tokens.pop(0)
                    if tokens[0] == ')':
                        tokens.pop(0)
                        return {key: sub_key}
            if len(tokens) > 0:
                tokens.pop(0)  
            return {key: nested}
        elif token == ')':
            return {}

    # Tokenize the string
    tokens = re.findall(r'\(|\)|\w+', s)
    return parse(tokens)

In [ ]:
def process_toppings(order_json):

    order = json.loads(order_json)
    processed_order = {"ORDER": {"PIZZAORDER": [], "DRINKORDER": []}}
    
    for pizza in order["ORDER"].get("PIZZAORDER", []):
        toppings = pizza.get("TOPPING", [])
        complex_toppings = pizza.get("COMPLEX_TOPPING", [])
        excluded = pizza.get("NOT", {})
        excluded_toppings = []
        tops=[]
        styles=[]
        excluded_styles=[]
        all_styles=[]
        for exc in excluded:
            tops.append(exc.get("TOPPING", []))
            if exc.get("STYLE", False):
                excluded_styles.append(exc.get("STYLE", {})[0])
        styles=pizza.get("STYLE", [])         
        for top in tops:
            for i in range(len(top)):
                excluded_toppings.append(top[i])
            
        if "NOT" in pizza:
            del pizza["NOT"]
        if "STYLE" in pizza: 
            del pizza["STYLE"]
        if "TOPPING" in pizza:
            del pizza["TOPPING"]
        if "COMPLEX_TOPPING" in pizza:
            del pizza["COMPLEX_TOPPING"]
        all_toppings = []
        
        for topping in toppings:
            all_toppings.append({
                "NOT": False,
                "Quantity": None,  
                "Topping": topping
            })
        for style in styles:
            all_styles.append({
                "NOT": False,
                "TYPE": style
            })
        for style in excluded_styles:
            all_styles.append({
                "NOT": True,
                "TYPE": style
            })
        
        for topping in excluded_toppings:
            all_toppings.append({
                "NOT": True,
                "Quantity": None,  
                "Topping": topping
            })
        for complex_topping in complex_toppings:
            complex_quantity = complex_topping.get("QUANTITY", None)
            complex_topping_list = complex_topping.get("TOPPING", [])
            for topping in complex_topping_list:
                all_toppings.append({
                    "NOT": False,
                    "Quantity": complex_quantity,
                    "Topping": topping
                })
        for drink in order["ORDER"].get("DRINKORDER", []):
            processed_order["ORDER"]["DRINKORDER"].append(drink)
        # Add the pizza order with updated AllTopping
        new_pizza_order = {**pizza, "AllTopping": all_toppings, "STYLE": all_styles}
        processed_order["ORDER"]["PIZZAORDER"].append(new_pizza_order)
    return json.dumps(processed_order,indent=4)

In [ ]:
def process_order(order_json):
    # Parse the input JSON
    order = json.loads(order_json)
    processed_order = {"ORDER": {"PIZZAORDER": [], "DRINKORDER": []}}
    
    for pizza in order["ORDER"].get("PIZZAORDER", []):
        number = pizza.get("NUMBER", None)
        size = pizza.get("SIZE", None)
        style = pizza.get("STYLE", None) 
        type_ = pizza.get("TYPE", None)  
        toppings = pizza.get("AllTopping", [])
        
        excluded_toppings = pizza.get("NOT", {}).get("TOPPING", [])
        toppings = [topping for topping in toppings if topping not in excluded_toppings]
        
        processed_order["ORDER"]["PIZZAORDER"].append({
            "NUMBER": number,
            "SIZE": size,
            "STYLE": style,
            "ALLTOPPING": toppings
        })
    for drink in order["ORDER"].get("DRINKORDER", []):
        number = drink.get("NUMBER", None)
        size = drink.get("SIZE", None)
        drink_type = drink.get("DRINKTYPE", None)
        container_type = drink.get("CONTAINERTYPE", None)

        processed_order["ORDER"]["DRINKORDER"].append({
            "NUMBER": number,
            "SIZE": size,
            "DRINKTYPE": drink_type,
            "CONTAINERTYPE": container_type
        })
    
    return json.dumps(processed_order, indent=4)

In [ ]:
def process_json(data):
    if isinstance(data, dict):
        # Process dictionary keys and values recursively
        return {key: process_json(value) for key, value in data.items()}
    elif isinstance(data, list):
        # Process each element in the list recursively
        return [process_json(item) for item in data]
    elif isinstance(data, str):
        # Convert string to lowercase and handle underscores
        return data.lower().replace('_', ' ')
    else:
        # Return data as-is if it's not a string, list, or dictionary
        return data

In [ ]:

for i, sequence in enumerate(pred_test):
    combined_tokens, combined_labels = combine_tokens(order_tokens_2[i], sequence)
    EXR_Format = Get_EXR_Format(combined_tokens, combined_labels)
    result = parse_to_dict(EXR_Format)
    json_string = json.dumps(result)
    processed_toppings = process_toppings(json_string)
    Json_Format=  process_order(processed_toppings)
    final_Json=process_json(json.loads(Json_Format))
    print(f"test {i}",final_Json)